In [2]:
import pandas as pd
import gmaps
import matplotlib.pyplot as plt
from citipy import citipy
import requests
import json
import matplotlib.pyplot as plt
# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [3]:
df = pd.read_csv("weather.csv")
df

,Unnamed: 0,city,latitude,lontitude,wind speed,cloudness,humidity,temp
0,0,atuona,-9.80,-139.03,16.15,0,74,81.81
1,1,mumbwa,-14.98,27.06,4.74,42,97,62.02
2,2,jatiroto,-7.88,111.12,2.10,40,94,78.80
3,3,butaritari,3.07,172.79,14.27,100,74,82.94
4,4,cidreira,-30.18,-50.21,22.35,96,82,75.27
...,...,...,...,...,...,...,...,...
496,496,laem sing,12.48,102.07,2.24,0,94,77.58
497,497,gaogou,34.02,119.19,9.40,23,44,42.57
498,498,kamuli,0.95,33.12,0.76,79,91,64.33
499,499,pimentel,-6.84,-79.93,20.80,75,29,66.20


In [4]:
# Store latitude and longitude in locations
locations = df[["latitude", "lontitude"]]

# Fill NaN values and convert to float
rating = df["humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level=1.25)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max(rating),
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
narrowed_city_df = df.loc[(df["temp"]>=70) & (df["temp"]<80) &(df["wind speed"]<=10)&(df["cloudness"]==0)].reset_index()

In [6]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": gkey
}

hotel=[]
hotel_city=[]
hotel_country=[]
hotel_lat=[]
hotel_lng=[]
for index, row in narrowed_city_df.iterrows():
    lat = narrowed_city_df["latitude"]
    lng = narrowed_city_df["lontitude"]
    
    params["location"] = f"{lat[index]},{lng[index]}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        name_address = requests.get(base_url, params=params)
        name_address = name_address.json()
        hotel.append(name_address["results"][0]["name"])
        hotel_lat.append(name_address["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(name_address["results"][0]["geometry"]["location"]["lng"])
        city = citipy.nearest_city(name_address["results"][0]["geometry"]["location"]["lat"],name_address["results"][0]["geometry"]["location"]["lng"])

        hotel_country.append(city.country_code)
        hotel_city.append(city.city_name)

    except:
        pass

In [7]:
hotel_df = pd.DataFrame({"hotel":hotel
                         ,"hotel_country":hotel_country
                         ,"hotel_city":hotel_city
                         ,"hotel_lat":hotel_lat
                        ,"hotel_lng":hotel_lng})

In [8]:

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{hotel_city}</dd>
<dt>Country</dt><dd>{hotel_country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["hotel_lat", "hotel_lng"]]

In [11]:

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure(center=(0, 0), zoom_level=1.25)
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
plt.savefig("heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>